In [ ]:
%run Imports.ipynb
from quantpy.tomography.interval import ConfidenceInterval, _pop_hidden_keys, Mode

import numpy as np
import scipy.stats as sts
import polytope as pc
import math

from enum import Enum, auto
from abc import ABC, abstractmethod
from scipy.interpolate import interp1d
from collections import Counter, defaultdict
from functools import partial
from einops import repeat, rearrange

from quantpy.geometry import hs_dst, trace_dst, if_dst
from quantpy.polytope import compute_polytope_volume, find_max_distance_to_polytope
from quantpy.qobj import Qobj
from quantpy.routines import (
    _left_inv, _vec2mat, _mat2vec,
    _matrix_to_real_tril_vec, _real_tril_vec_to_matrix,
)
from quantpy.stats import l2_first_moment, l2_mean, l2_second_moment, l2_variance
from quantpy.tomography.interval import ConfidenceInterval, Mode
from quantpy.tomography.polytopes.verification import test_qpt, test_qst

### QST number of qubits

In [ ]:
conf_levels = np.concatenate((np.arange(0.1, 0.9, 0.1), np.arange(0.9, 1, 0.01)))
n_qubits_list = range(1, 6)
results = []
n_trials = 10000
n_measurements = 10 ** 4
for i in n_qubits_list:
    state = qp.qobj.GHZ(i)
    results.append(test_qst(state, conf_levels, n_measurements, n_trials))

In [ ]:
obj = {'cl': conf_levels, 'results': results}
with open('../results/states_qubits_10k.pkl', 'wb') as file:
    pickle.dump(obj, file)

In [ ]:
with open('../results/states_qubits_10k.pkl', 'rb') as file:
    obj = pickle.load(file)
results = obj['results']

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('$f_{\\rm fail}$')
plt.xscale('log')
plt.yscale('log')
plt.grid(which='both')
plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10), '--k')
for n, results_for_qubit in enumerate(results):
    plt.plot(1 - conf_levels, 1 - results_for_qubit, '-o', label=f'$N={n+1}$', linewidth=3)
plt.legend()
plt.figtext(0.02, 0.9, "a)")
plt.savefig('../imgs/fig1a.pdf')

### QST number of measurements

In [ ]:
conf_levels = np.concatenate((np.arange(0.1, 0.9, 0.1), np.arange(0.9, 1, 0.01)))
n_meas_list = [int(n) for n in np.logspace(2, 7, 6)]
results = np.zeros((len(n_meas_list), len(conf_levels)))
n_trials = 10000
for i, n_measurements in enumerate(n_meas_list):
    state = qp.qobj.GHZ(1)
    dim = 2 ** state.n_qubits
    tmg = qp.StateTomograph(state)
    tmg.experiment(n_measurements)
    state_hat = tmg.point_estimate()
    EPS = 1e-15

    povm_matrix = (np.reshape(tmg.povm_matrix * tmg.n_measurements[:, None, None]
                              / np.sum(tmg.n_measurements),
                              (-1, tmg.povm_matrix.shape[-1]))
                   * tmg.povm_matrix.shape[0])
    A = np.ascontiguousarray(povm_matrix[:, 1:]) * dim
    polytope_prod = A @ state.bloch[1:]
    
    for _ in tqdm(range(n_trials)):
        tmg = qp.StateTomograph(state)
        tmg.experiment(n_measurements)
        frequencies = np.clip(tmg.raw_results / tmg.n_measurements[:, None], EPS, 1 - EPS)
        for j, cl in enumerate(conf_levels):
            delta = WangInterval._count_delta(cl, frequencies, tmg.n_measurements)
            b = np.clip(np.hstack(frequencies) + delta, EPS, 1 - EPS) - povm_matrix[:, 0]
            if np.min(b - polytope_prod) > -EPS:
                results[i, j] += 1
results /= n_trials

In [ ]:
with open('../results/states_meas.pkl', 'rb') as file:
    obj = pickle.load(file)
results = obj['results']

In [ ]:
obj = {'cl': conf_levels, 'results': new_results}
with open('../results/states_meas.pkl', 'wb') as file:
    pickle.dump(obj, file)

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('$f_{\\rm fail}$')
plt.xscale('log')
plt.yscale('log')
plt.grid(which='both')
plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10), '--k')
for n, row in enumerate(results[:-2]):
    plt.plot(1 - conf_levels, 1 - row, '-o', label=f'$n=10^{n+2}$', linewidth=3)
plt.legend()
plt.figtext(0.02, 0.9, "b)")
plt.savefig('../imgs/fig1b.pdf')

### QPT number of qubits

In [ ]:
conf_levels = np.concatenate((np.arange(0.1, 0.9, 0.1), np.arange(0.9, 1, 0.01)))
n_qubits_list = range(1, 4)
results = []
n_trials = 10000
n_measurements = 10 ** 4
for i in range(1, 4):
    channel = qp.channel.depolarizing(p=0.1, n_qubits=i)
    results.append(test_qpt(channel, conf_levels, n_measurements, n_trials))

In [ ]:
obj = {'cl': conf_levels, 'results': results}
with open('../results/processes_qubits_10k.pkl', 'wb') as file:
    pickle.dump(obj, file)

In [ ]:
with open('../results/processes_qubits_10k.pkl', 'rb') as file:
    obj = pickle.load(file)
results = obj['results']

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('$f_{\\rm fail}$')
plt.xscale('log')
plt.yscale('log')
plt.grid(which='both')
plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10), '--k')
for n, results_row in enumerate(results):
    plt.plot(1 - conf_levels, 1 - results_row, '-o', label=f'$N={n+1}$', linewidth=3)
plt.legend()
plt.figtext(0.02, 0.9, "c)")
plt.savefig('../imgs/fig1c.pdf')

### QPT number of measurements

In [ ]:
conf_levels = np.concatenate((np.arange(0.1, 0.9, 0.1), np.arange(0.9, 1, 0.01)))
n_meas_list = [int(n) for n in np.logspace(2, 5, 4)]
results = []
n_trials = 10000
for n_measurements in n_meas_list:
    channel = qp.channel.depolarizing(p=0.1, n_qubits=1)
    results.append(test_qpt(channel, conf_levels, n_measurements, n_trials))

In [ ]:
obj = {'cl': conf_levels, 'results': results}
with open('../results/processes_meas.pkl', 'wb') as file:
    pickle.dump(obj, file)

In [ ]:
with open('../results/processes_meas.pkl', 'rb') as file:
    obj = pickle.load(file)
results = obj['results']

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('$f_{\\rm fail}$')
plt.xscale('log')
plt.yscale('log')
plt.grid(which='both')
plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10), '--k')
for n, row in enumerate(results):
    plt.plot(1 - conf_levels, 1 - row, '-o', label=f'$n=10^{n+2}$', linewidth=3)
plt.legend()
plt.figtext(0.02, 0.9, "d)")
plt.savefig('../imgs/fig1d.pdf')